## Cleanup OpenShift Environment

⚠️ **WARNING**: This notebook is **OPTIONAL** and should only be used if you want to completely remove all deployed resources from your OpenShift cluster.

**This cleanup is NOT required** to complete the demo. You can keep the resources running for future use.

This notebook will:
1. Delete deployed MCP resources (MCPServer instances)
2. Delete the MCP Registry instance
3. Delete the PostgreSQL cluster
4. Uninstall the ToolHive operator and CRDs


### Delete MCP Server Resources

Remove any deployed MCP server instances.


In [1]:
# List all MCPServer instances
!oc get mcpserver -A


No resources found


In [3]:
# Delete the Kubernetes MCP Server (if deployed)
!oc delete -f ../manifests/kubernetes-mcp-server.yaml


Error from server (NotFound): error when deleting "../manifests/kubernetes-mcp-server.yaml": mcpservers.toolhive.stacklok.dev "io-github-stacklok-kubernetes-mcp-server" not found


### Delete MCP Registry Instance

Remove the deployed MCP Registry and its associated resources.


In [4]:
# Delete the demo-registry instance
!oc delete -f ../manifests/demo-registry.yaml


secret "demo-registry-db-password" deleted
mcpregistry.toolhive.stacklok.dev "demo-registry" deleted


In [5]:
# Delete registry-related ConfigMaps
!oc delete cm rh-catalog-mcp rh-partners-mcp rh-one-mcp --ignore-not-found=true


configmap "rh-catalog-mcp" deleted
configmap "rh-partners-mcp" deleted
configmap "rh-one-mcp" deleted


In [6]:
# Verify registry resources are deleted
!oc get mcpregistry -A


No resources found


### Delete PostgreSQL Cluster

Remove the PostgreSQL cluster created for the registry database.


In [7]:
# List PostgreSQL clusters
!oc get cluster -A


NAMESPACE         NAME      AGE   INSTANCES   READY   STATUS                     PRIMARY
toolhive-system   demo-db   14h   1           1       Cluster in healthy state   demo-db-1


In [9]:
# Delete the demo-db PostgreSQL cluster
!oc delete -f ../manifests/psql-db.yaml


cluster.postgresql.cnpg.io "demo-db" deleted


In [10]:
# Wait for cluster deletion to complete
!oc wait --for=delete cluster/demo-db -n toolhive-system --timeout=300s || echo "Cluster already deleted or not found"


### Uninstall ToolHive Operator

Remove the ToolHive operator using Helm uninstall. This will remove both the operator and CRDs.


In [11]:
# List Helm releases in toolhive-system namespace
!helm list -n toolhive-system


NAME                  	NAMESPACE      	REVISION	UPDATED                             	STATUS  	CHART                        	APP VERSION
toolhive-operator     	toolhive-system	2       	2025-12-22 18:13:14.771158 +0100 CET	deployed	toolhive-operator-0.5.17     	v0.6.15    
toolhive-operator-crds	toolhive-system	6       	2025-12-22 18:01:05.390522 +0100 CET	deployed	toolhive-operator-crds-0.0.88	0.0.1      


In [12]:
# Uninstall the ToolHive operator
!helm uninstall toolhive-operator -n toolhive-system || echo "Operator release not found"


release "toolhive-operator" uninstalled


In [14]:
# Uninstall the ToolHive operator CRDs
!helm uninstall toolhive-operator-crds -n toolhive-system || echo "CRDs release not found"
!oc delete $(oc get crd | grep mcp)


Error: uninstall: Release not loaded: toolhive-operator-crds: release: not found
CRDs release not found
Error from server (NotFound): mcpexternalauthconfigs.toolhive.stacklok.dev "2025-12-22T14:36:13Z" not found
Error from server (NotFound): mcpexternalauthconfigs.toolhive.stacklok.dev "mcpgroups.toolhive.stacklok.dev" not found
Error from server (NotFound): mcpexternalauthconfigs.toolhive.stacklok.dev "2025-12-22T14:36:13Z" not found
Error from server (NotFound): mcpexternalauthconfigs.toolhive.stacklok.dev "mcpregistries.toolhive.stacklok.dev" not found
Error from server (NotFound): mcpexternalauthconfigs.toolhive.stacklok.dev "2025-12-22T14:36:13Z" not found
Error from server (NotFound): mcpexternalauthconfigs.toolhive.stacklok.dev "mcpremoteproxies.toolhive.stacklok.dev" not found
Error from server (NotFound): mcpexternalauthconfigs.toolhive.stacklok.dev "2025-12-22T14:36:13Z" not found
Error from server (NotFound): mcpexternalauthconfigs.toolhive.stacklok.dev "mcpservers.toolhive.

### Verify CRDs Removal

Check that ToolHive CRDs have been removed. **Note**: CRDs are removed when uninstalling the `toolhive-operator-crds` Helm release.


In [15]:
# Verify ToolHive CRDs have been removed
!oc get crd | grep toolhive || echo "No ToolHive CRDs found (already removed)"


mcpexternalauthconfigs.toolhive.stacklok.dev                                       2025-12-22T14:36:13Z
mcpgroups.toolhive.stacklok.dev                                                    2025-12-22T14:36:13Z
mcpregistries.toolhive.stacklok.dev                                                2025-12-22T14:36:13Z
mcpremoteproxies.toolhive.stacklok.dev                                             2025-12-22T14:36:13Z
mcpservers.toolhive.stacklok.dev                                                   2025-12-22T14:36:13Z
mcptoolconfigs.toolhive.stacklok.dev                                               2025-12-22T14:36:13Z
virtualmcpcompositetooldefinitions.toolhive.stacklok.dev                           2025-12-22T14:36:13Z
virtualmcpservers.toolhive.stacklok.dev                                            2025-12-22T14:36:13Z


### Cleanup Namespace (Optional)

If you want to completely remove the `toolhive-system` namespace and all its resources:


In [17]:
# List remaining resources in toolhive-system namespace
!oc get all -n toolhive-system


NAME                                           READY   STATUS    RESTARTS   AGE
pod/cnpg-controller-manager-846579bb9d-cpkdk   1/1     Running   0          11d

NAME                                      TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)   AGE
service/cnpg-controller-manager-service   ClusterIP   172.30.35.183   <none>        443/TCP   18d
service/cnpg-webhook-service              ClusterIP   172.30.41.125   <none>        443/TCP   18d

NAME                                      READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/cnpg-controller-manager   1/1     1            1           18d

NAME                                                 DESIRED   CURRENT   READY   AGE
replicaset.apps/cnpg-controller-manager-5b7d9c5f5b   0         0         0       11d
replicaset.apps/cnpg-controller-manager-846579bb9d   1         1         1       11d


In [ ]:
# Delete the toolhive-system namespace (WARNING: This deletes everything in the namespace)
# Uncomment the line below only if you want to completely remove the namespace
# !oc delete namespace toolhive-system

## Cleanup Complete

All ToolHive resources have been removed from your OpenShift cluster:

- ✅ MCP Server instances deleted
- ✅ MCP Registry instance deleted
- ✅ PostgreSQL cluster deleted
- ✅ ToolHive operator uninstalled
- ✅ Custom Resource Definitions removed

**Note**: The `toolhive-system` namespace may still exist. If you want to remove it completely, uncomment the command in the previous cell.

To redeploy, simply run the notebooks in order:
1. `01-setup.ipynb` - Install ToolHive operator
2. `02-deploy-registry.ipynb` - Deploy MCP Registry
3. `03-deploy-and-discover-mcp-server.ipynb` - Deploy and discover MCP servers
